# Linear Algebra Assignment 4
## 110062219 翁君牧

In [1]:
# !export OMP_NUM_THREADS=$(nproc)
# !export MKL_NUM_THREADS=$(nproc)

import sys

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

try:
    from sklearnex import patch_sklearn
    patch_sklearn()
except Exception as e:
    print(e, file=sys.stderr)
    
try:
    %load_ext autotime
except:
    pass

time: 208 µs (started: 2023-01-02 18:09:25 +08:00)


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
# ---------- using linear least square ----------------
def LSTSQDistance(spam, ham, test):
    X1 = np.linalg.lstsq(spam, test, rcond=None)[0]
    R1 = np.dot(spam, X1) - test
    X2 = np.linalg.lstsq(ham, test, rcond=None)[0]
    R2 = np.dot(ham, X2) - test
    [m, n] = R1.shape
    pos = 0
    neg = 0
    for i in range(n):
        # distance to the spam subspace
        d1 = np.linalg.norm(R1[:,i])
        # distance to the ham subspace
        d2 = np.linalg.norm(R2[:,i])
        if d1 >= d2:
            # not a spam
            neg = neg + 1
        else:
            # is a spam
            pos = pos + 1

    return pos, neg

time: 761 µs (started: 2023-01-02 18:09:25 +08:00)


In [3]:
# ---------- main program --------------------
emails = pd.read_csv("emails.csv")
df = pd.concat([emails, pd.read_csv("A4_110062219.csv")], ignore_index=True)
spam = df['spam']
fold = df['fold']

# initialize
cv = TfidfVectorizer(stop_words='english', max_features=20000, token_pattern=r"(?u)\b[a-zA-Z]\w+\b") 
dt_matrix = cv.fit_transform(df['text'])
[m, n] = dt_matrix.shape

# For each email, we have two tags: spam and fold.
# Spam = 1 means the email is a spam; spam = 0 means the email is not a spam
# And for each email, it also belongs to a fold.
# In this file, there are 5 fold, numbered from 0 to 4.
# In this example, we will use fold 0-3 as training data and fold 4 as test data.

spam_train = (dt_matrix[[i for i in range(m) if fold[i] != 4 and spam[i]], :]).toarray().transpose()
spam_test = (dt_matrix[[i for i in range(m) if fold[i] == 4 and spam[i]], :]).toarray().transpose()
ham_train = (dt_matrix[[i for i in range(m) if fold[i] != 4 and not spam[i]], :]).toarray().transpose()
ham_test = (dt_matrix[[i for i in range(m) if fold[i] == 4 and not spam[i]], :]).toarray().transpose()

/home/nevikw39/.conda/envs/linalg/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1640: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


time: 923 ms (started: 2023-01-02 18:09:25 +08:00)


In [4]:
# Compute the confusion matrix
p1, n1 = LSTSQDistance(spam_train, ham_train, spam_test)
p2, n2 = LSTSQDistance(spam_train, ham_train, ham_test)
print(p1, n1)
print(p2, n2)

208 74
1 861
time: 20.1 s (started: 2023-01-02 18:09:26 +08:00)


## Q3

In [5]:
for j in range(4):
    spam_train = (dt_matrix[[i for i in range(m) if fold[i] != j and spam[i]], :]).toarray().transpose()
    spam_test = (dt_matrix[[i for i in range(m) if fold[i] == j and spam[i]], :]).toarray().transpose()
    ham_train = (dt_matrix[[i for i in range(m) if fold[i] != j and not spam[i]], :]).toarray().transpose()
    ham_test = (dt_matrix[[i for i in range(m) if fold[i] == j and not spam[i]], :]).toarray().transpose()

    # Compute the confusion matrix
    tp, fn = LSTSQDistance(spam_train, ham_train, spam_test)
    fp, tn = LSTSQDistance(spam_train, ham_train, ham_test)

    print(np.array([[tp, fp], [fn, tn]]))
    print("Acc =", (tp + tn) / (tp + fn + tn + fp))
    print("  P =", tp / (tp + fp))
    print("  R =", tp / (tp + fn))

[[202   0]
 [ 67 870]]
Acc = 0.9411764705882353
  P = 1.0
  R = 0.7509293680297398
[[220   0]
 [ 68 877]]
Acc = 0.9416309012875537
  P = 1.0
  R = 0.7638888888888888
[[199   1]
 [ 70 877]]
Acc = 0.9380993897122929
  P = 0.995
  R = 0.7397769516728625
[[194   0]
 [ 71 871]]
Acc = 0.9375
  P = 1.0
  R = 0.7320754716981132
time: 1min 15s (started: 2023-01-02 18:09:46 +08:00)


## Q4

In [6]:
def SVDDistance(spam: np.ndarray, ham: np.ndarray, test1: np.ndarray, test2: np.ndarray) -> tuple[int, int, int, int]:
    U1, S1, V1 = np.linalg.svd(spam, full_matrices=False)
    basis1 = U1[:, np.flatnonzero(S1)]

    U2, S2, V2 = np.linalg.svd(ham, full_matrices=False)
    basis2 = U2[:, np.flatnonzero(S2)]
    
    lst = []
    for test in [test1, test2]:
        proj1 = np.dot(basis1, np.dot(basis1.T, test))
        proj2 = np.dot(basis2, np.dot(basis2.T, test))

        r1 = test - proj1
        r2 = test - proj2

        pos = 0
        neg = 0

        for i in range(test.shape[1]):
            d1 = np.linalg.norm(r1[:, i])
            d2 = np.linalg.norm(r2[:, i])
            if d1 <= d2:
                pos += 1
            else:
                neg += 1
        lst.extend((pos, neg))

    return (*lst,)

time: 954 µs (started: 2023-01-02 18:11:02 +08:00)


In [7]:
spam_train = (dt_matrix[[i for i in range(m) if fold[i] != 4 and spam[i]], :]).toarray().transpose()
spam_test = (dt_matrix[[i for i in range(m) if fold[i] == 4 and spam[i]], :]).toarray().transpose()
ham_train = (dt_matrix[[i for i in range(m) if fold[i] != 4 and not spam[i]], :]).toarray().transpose()
ham_test = (dt_matrix[[i for i in range(m) if fold[i] == 4 and not spam[i]], :]).toarray().transpose()

# Compute the confusion matrix
tp, fn, fp, tn = SVDDistance(spam_train, ham_train, spam_test, ham_test)

print(np.array([[tp, fp], [fn, tn]]))
print("Acc =", (tp + tn) / (tp + fn + tn + fp))
print("  P =", tp / (tp + fp))
print("  R =", tp / (tp + fn))

[[206   1]
 [ 76 861]]
Acc = 0.9326923076923077
  P = 0.9951690821256038
  R = 0.7304964539007093
time: 5.24 s (started: 2023-01-02 18:11:02 +08:00)


## Q5

In [8]:
def SVDDistance_LRA(spam: np.ndarray, ham: np.ndarray, test1: np.ndarray, test2: np.ndarray, k1: int, k2: int) -> tuple[int, int, int, int]:
    U1, S1, V1 = np.linalg.svd(spam, full_matrices=False)
    basis1 = U1[:, np.flatnonzero(S1[:k1])]

    U2, S2, V2 = np.linalg.svd(ham, full_matrices=False)
    basis2 = U2[:, np.flatnonzero(S2[:k2])]
    
    lst = []
    for test in [test1, test2]:
        proj1 = np.dot(basis1, np.dot(basis1.T, test))
        proj2 = np.dot(basis2, np.dot(basis2.T, test))

        r1 = test - proj1
        r2 = test - proj2

        pos = 0
        neg = 0

        for i in range(test.shape[1]):
            d1 = np.linalg.norm(r1[:, i])
            d2 = np.linalg.norm(r2[:, i])
            if d1 <= d2:
                pos += 1
            else:
                neg += 1
        lst.extend((pos, neg))

    return (*lst,)

time: 940 µs (started: 2023-01-02 18:11:07 +08:00)


In [9]:
lst = []

for k1 in [200, 300, 600]:
    for k2 in [100, 200, 300, 600, 1200]:
        print(k1, k2)
        
        %time tp, fn, fp, tn = SVDDistance_LRA(spam_train, ham_train, spam_test, ham_test, k1, k2)

        print(np.array([[tp, fp], [fn, tn]]))
        print("Acc =", (tp + tn) / (tp + fn + tn + fp))
        print("  P =", tp / (tp + fp))
        print("  R =", tp / (tp + fn))
        
        lst.append([k1, k2, fn, fp])

np.savetxt("A4_110062219.dat", lst, fmt="%d")

200 100
CPU times: user 4min 9s, sys: 7.25 s, total: 4min 17s
Wall time: 4.77 s
[[279  31]
 [  3 831]]
Acc = 0.9702797202797203
  P = 0.9
  R = 0.9893617021276596
200 200
CPU times: user 3min 53s, sys: 6.84 s, total: 4min
Wall time: 4.47 s
[[276  10]
 [  6 852]]
Acc = 0.986013986013986
  P = 0.965034965034965
  R = 0.9787234042553191
200 300
CPU times: user 3min 46s, sys: 6.57 s, total: 3min 52s
Wall time: 4.32 s
[[272   2]
 [ 10 860]]
Acc = 0.9895104895104895
  P = 0.9927007299270073
  R = 0.9645390070921985
200 600
CPU times: user 3min 53s, sys: 7.64 s, total: 4min 1s
Wall time: 4.48 s
[[253   1]
 [ 29 861]]
Acc = 0.9737762237762237
  P = 0.9960629921259843
  R = 0.8971631205673759
200 1200
CPU times: user 3min 52s, sys: 7.07 s, total: 3min 59s
Wall time: 4.49 s
[[218   0]
 [ 64 862]]
Acc = 0.9440559440559441
  P = 1.0
  R = 0.7730496453900709
300 100
CPU times: user 3min 53s, sys: 7.42 s, total: 4min
Wall time: 4.47 s
[[281  51]
 [  1 811]]
Acc = 0.9545454545454546
  P = 0.846385542

## Q6

In [10]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize

# from dask.distributed import Client
# import joblib


# client = Client()


def NMFDistance(spam: np.ndarray, ham: np.ndarray, test1: np.ndarray, test2: np.ndarray, k1: int, k2: int) -> tuple[int, int, int, int]:
    model1 = NMF(n_components=k1, init="nndsvd", solver="mu")
    # with joblib.parallel_backend('dask'):
    W1 = model1.fit_transform(normalize(spam))
    # H1 = model1.components_
    
    model2 = NMF(n_components=k2, init="nndsvd", solver="mu")
    # with joblib.parallel_backend('dask'):
    W2 = model2.fit_transform(normalize(ham))
    # H2 = model2.components_
    
    lst = []
    for test in [test1, test2]:
        proj1 = np.dot(W1, np.dot(W1.T, test))
        proj2 = np.dot(W2, np.dot(W2.T, test))

        r1 = test - proj1
        r2 = test - proj2

        pos = 0
        neg = 0

        for i in range(test.shape[1]):
            d1 = np.linalg.norm(r1[:, i])
            d2 = np.linalg.norm(r2[:, i])
            if d1 <= d2:
                pos += 1
            else:
                neg += 1
        lst.extend((pos, neg))

    return (*lst,)


time: 1.06 ms (started: 2023-01-02 18:12:14 +08:00)


In [11]:
for k1 in [200, 300, 600]:
    for k2 in [100, 200, 300, 600, 1200]:
        print(k1, k2)

        %time tp, fn, fp, tn = NMFDistance(spam_train, ham_train, spam_test, ham_test, k1, k2)

        print(np.array([[tp, fp], [fn, tn]]))
        print("Acc =", (tp + tn) / (tp + fn + tn + fp))
        print("  P =", tp / (tp + fp))
        print("  R =", tp / (tp + fn))

200 100


/home/nevikw39/.conda/envs/linalg/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1466: UserWarning: The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.
  warnings.warn(
/home/nevikw39/.conda/envs/linalg/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1466: UserWarning: The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.
  warnings.warn(


CPU times: user 7min 31s, sys: 13.4 s, total: 7min 44s
Wall time: 10 s
[[ 85 551]
 [197 311]]
Acc = 0.34615384615384615
  P = 0.13364779874213836
  R = 0.30141843971631205
200 200


/home/nevikw39/.conda/envs/linalg/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1466: UserWarning: The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.
  warnings.warn(
/home/nevikw39/.conda/envs/linalg/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1466: UserWarning: The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.
  warnings.warn(


CPU times: user 12min 33s, sys: 28.2 s, total: 13min 1s
Wall time: 16.3 s
[[ 74 283]
 [208 579]]
Acc = 0.5708041958041958
  P = 0.20728291316526612
  R = 0.2624113475177305
200 300


...

CPU times: user 25min 50s, sys: 48 s, total: 26min 38s
Wall time: 31.4 s
[[175 306]
 [107 556]]
Acc = 0.638986013986014
  P = 0.36382536382536385
  R = 0.6205673758865248
600 600


/home/nevikw39/.conda/envs/linalg/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1466: UserWarning: The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.
  warnings.warn(
/home/nevikw39/.conda/envs/linalg/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1466: UserWarning: The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.
  warnings.warn(


CPU times: user 32min 39s, sys: 55.9 s, total: 33min 35s
Wall time: 39.1 s
[[158  73]
 [124 789]]
Acc = 0.8277972027972028
  P = 0.683982683982684
  R = 0.5602836879432624
600 1200


/home/nevikw39/.conda/envs/linalg/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1466: UserWarning: The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.
  warnings.warn(
/home/nevikw39/.conda/envs/linalg/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1466: UserWarning: The multiplicative update ('mu') solver cannot update zeros present in the initialization, and so leads to poorer results when used jointly with init='nndsvd'. You may try init='nndsvda' or init='nndsvdar' instead.
  warnings.warn(


CPU times: user 1h 1min 12s, sys: 2min 13s, total: 1h 3min 26s
Wall time: 1min 12s
[[144  49]
 [138 813]]
Acc = 0.8365384615384616
  P = 0.7461139896373057
  R = 0.5106382978723404
time: 7min 36s (started: 2023-01-02 18:12:14 +08:00)
